In [5]:
import psycopg2
from flask import Flask

app = Flask(__name__)

@app.route('/data')
def get_data():
    # 클라이언트의 DB에 접속합니다.
    conn = psycopg2.connect(
        host="eda-client",
        database="test_cli",
        user="postgres",
        password="postgres",
        port="5432"
    )

    # 데이터를 수집합니다.
    cur = conn.cursor()
    cur.execute("SELECT * FROM users")
    data = cur.fetchall()

    # 응답을 생성합니다.
    response = "\n".join(str(row) for row in data)
    return response

if __name__ == '__main__':
    app.run(host="eda", port=80)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://eda:80
Press CTRL+C to quit
172.180.9.2 - - [25/Apr/2023 17:46:30] "GET /data HTTP/1.1" 200 -


In [21]:
import os
import psycopg2
from flask import Flask, request
from apscheduler.schedulers.background import BackgroundScheduler
import uuid

app = Flask(__name__)
scheduler = BackgroundScheduler()

@app.route('/config', methods=['POST'])
def get_config():
    # Get database configuration data from the client request
    data = request.get_json()
    db_type = data['db_type']
    db_host = data['db_host']
    db_port = data['db_port']
    db_name = data['db_name']
    db_user = data['db_user']
    db_password = data['db_password']
    
    server_conn = psycopg2.connect(
        host='localhost',
        database='eda',
        user='postgres',
        password='postgres'
    )
    
    # Add collect_metrics function to the scheduler
    # Check if the input DB configuration already exists in the database
    cur = server_conn.cursor()
    cur.execute("""
        SELECT id
        FROM db_config
        WHERE db_type = %s AND db_host = %s AND db_port = %s AND db_name = %s AND db_user = %s;
    """, (db_type, db_host, db_port, db_name, db_user))
    result = cur.fetchone()

    if result:
        # If the input DB configuration already exists in the database, use the existing ID
        db_id = result[0]
    else:
        # If the input DB configuration does not exist in the database, generate a new ID
        db_id = str(uuid.uuid4())
        print(db_id, db_type, db_host, db_port, db_name, db_user, db_password)
        # Store the new DB ID and configuration in the database
        cur.execute("""
            INSERT INTO db_config (id, db_type, db_host, db_port, db_name, db_user, db_password)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (db_id, db_type, db_host, db_port, db_name, db_user, db_password))
        server_conn.commit()

    # Return the DB ID to the client
    # scheduler.add_job(collect_metrics, 'interval', minutes=1)
    # scheduler.start()

    # Return a success response
    return f"{db_id}Database configuration data received successfully!"

def collect_metrics():
    # Get database configuration from environment variables
    db_host = os.environ.get('DB_HOST')
    db_port = os.environ.get('DB_PORT')
    db_name = os.environ.get('DB_NAME')
    db_user = os.environ.get('DB_USER')
    db_password = os.environ.get('DB_PASSWORD')

    # Connect to the client database and collect metrics
    conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Run queries to collect metrics and store them in a file or database

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=80)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:80
 * Running on http://172.180.9.3:80
Press CTRL+C to quit
172.180.9.2 - - [26/Apr/2023 14:33:34] "POST /config HTTP/1.1" 200 -


{'db_type': 'postgres', 'db_host': 'eda-client', 'db_name': 'test_cli', 'db_user': 'postgres', 'db_password': 'postgres', 'db_port': '5432', 'interval': 10}
34c48061-022a-4224-9ce7-8a3917ed40d5 postgres eda-client 5432 test_cli postgres postgres
